# Exploratory Analysis Part 1


## Importing libraries and combining the dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
sb.set()
from sklearn import preprocessing
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from collections import Counter
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
mmScaler = MinMaxScaler()
sScaler = StandardScaler()
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
df1 = pd.read_csv('hdb_nearest_bus - merged.csv')
# print the dataset 
len(df1['Postal'].unique())
df1

,Unnamed: 0,Unnamed: 1,BusStopCode,RoadName,Description,Latitude,Longitude,UpDown,LeftRight,HDB_Bus_Distance,Postal
0,26103.0,457,43581,Bt Batok West Ave 5,Blk 347,1.362519,103.751384,-15,0,95.032670,650383
1,NaN,458,43589,Bt Batok West Ave 5,Opp Blk 347,1.362273,103.751532,-15,0,76.282252,650383
2,38564.0,457,43581,Bt Batok West Ave 5,Blk 347,1.362519,103.751384,-15,0,100.970052,650384
3,NaN,458,43589,Bt Batok West Ave 5,Opp Blk 347,1.362273,103.751532,-15,0,103.767223,650384
4,38593.0,470,43649,Bt Batok West Ave 7,HomeTeamNS,1.365458,103.750602,-15,1,185.836015,650388
...,...,...,...,...,...,...,...,...,...,...,...
17531,62564.0,360,46521,Woodlands Ave 3,Marsiling Stn,1.432378,103.774293,-10,14,77.377975,730163
17532,NaN,361,46529,Woodlands Ave 3,Opp Marsiling Stn,1.432082,103.774597,-10,14,122.842369,730163
17533,28646.0,313,60031,Kallang Bahru,BEF GEYLANG BAHRU STN,1.320410,103.870078,9,-8,197.821207,330072
17534,NaN,339,60221,Geylang Bahru,BLK 73/GEYLANG BAHRU STN,1.322124,103.870814,9,-8,90.081765,330072


In [3]:
df1 = df1[['Postal', 'BusStopCode', 'HDB_Bus_Distance']]

In [4]:
df1['No_Bus_Stops'] = 1
df1['No_Bus_Stops']=df1.groupby('Postal')['Postal'].transform('count')
df1 = df1.drop(['BusStopCode', 'HDB_Bus_Distance'], axis =1)
df1

,Postal,No_Bus_Stops
0,650383,4
1,650383,4
2,650384,4
3,650384,4
4,650388,2
...,...,...
17531,730163,2
17532,730163,2
17533,330072,3
17534,330072,3


In [5]:
df2 = pd.read_csv('hdb_to_mrt_all.csv')
# print the dataset 
df2.columns

C:\Users\samik\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3203: DtypeWarning: Columns (38) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Index(['Unnamed: 0', 'month', 'town', 'flat_type', 'blk_no', 'street',
       'storey_range', 'floor_area_sqm', 'flat_model', 'lease_commence_date',
       'remaining_lease', 'resale_price', 'max_floor_lvl', 'year_completed',
       'residential', 'commercial', 'market_hawker', 'miscellaneous',
       'multistorey_carpark', 'precinct_pavilion', 'bldg_contract_town',
       'total_dwelling_units', '1room_sold', '2room_sold', '3room_sold',
       '4room_sold', '5room_sold', 'exec_sold', 'multigen_sold',
       'studio_apartment_sold', '1room_rental', '2room_rental', '3room_rental',
       'other_room_rental', 'lat_hdb', 'lng_hdb', 'building', 'addr', 'Postal',
       'SUBZONE_NO', 'SUBZONE_N', 'SUBZONE_C', 'PLN_AREA_N', 'PLN_AREA_C',
       'REGION_N', 'REGION_C', 'MRT_NAME', 'MRT_ID', 'line', 'no', 'lng_mrt',
       'lat_mrt', 'mahattan_distance', 'mrt_cbd_dist', 'mrt_cbd_time',
       'hdb_cbd_distance', 'hdb_cbd_time', 'hdb_to_mrtdist', 'sgd_persqm'],
      dtype='object')

In [6]:
combined = df2.merge(df1, on=['Postal'], how='left')

In [7]:
combined.loc[combined['No_Bus_Stops'].isnull(), 'No_Bus_Stops']=0
combined

,Unnamed: 0,month,town,flat_type,blk_no,street,storey_range,floor_area_sqm,flat_model,lease_commence_date,...,lng_mrt,lat_mrt,mahattan_distance,mrt_cbd_dist,mrt_cbd_time,hdb_cbd_distance,hdb_cbd_time,hdb_to_mrtdist,sgd_persqm,No_Bus_Stops
0,26103,2017-03,BUKIT BATOK,4 ROOM,383,BT BATOK WEST AVE 5,07 TO 09,92.0,Model A,2005,...,103.761535,1.379002,20620.13889,18877.9,23.801667,19015.7,22.508333,2885.65,4836.96,0.0
1,26104,2017-04,BUKIT BATOK,4 ROOM,383,BT BATOK WEST AVE 5,19 TO 21,90.0,Model A,2005,...,103.761535,1.379002,20620.13889,18877.9,23.801667,19015.7,22.508333,2885.65,5143.20,0.0
2,26105,2017-05,BUKIT BATOK,4 ROOM,383,BT BATOK WEST AVE 5,01 TO 03,90.0,Model A,2005,...,103.761535,1.379002,20620.13889,18877.9,23.801667,19015.7,22.508333,2885.65,4388.89,0.0
3,26106,2017-09,BUKIT BATOK,5 ROOM,383,BT BATOK WEST AVE 5,10 TO 12,115.0,Improved,2005,...,103.761535,1.379002,20620.13889,18877.9,23.801667,19015.7,22.508333,2885.65,4782.61,0.0
4,26107,2017-12,BUKIT BATOK,4 ROOM,383,BT BATOK WEST AVE 5,10 TO 12,90.0,Model A,2005,...,103.761535,1.379002,20620.13889,18877.9,23.801667,19015.7,22.508333,2885.65,4588.89,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184928,16451,2018-05,CHOA CHU KANG,4 ROOM,152,JLN TECK WHYE,07 TO 09,101.0,Model A,1998,...,103.758034,1.378619,20966.20658,18154.7,22.730000,18297.9,23.383333,23.24,3287.13,0.0
184929,16455,2018-08,CHOA CHU KANG,4 ROOM,152,JLN TECK WHYE,13 TO 15,101.0,Model A,1998,...,103.758034,1.378619,20966.20658,18154.7,22.730000,18297.9,23.383333,23.24,3712.87,0.0
184930,16459,2018-11,CHOA CHU KANG,4 ROOM,152,JLN TECK WHYE,01 TO 03,100.0,Model A,1998,...,103.758034,1.378619,20966.20658,18154.7,22.730000,18297.9,23.383333,23.24,3250.00,0.0
184931,16463,2020-03,CHOA CHU KANG,4 ROOM,152,JLN TECK WHYE,04 TO 06,100.0,Model A,1998,...,103.758034,1.378619,20966.20658,18154.7,22.730000,18297.9,23.383333,23.24,3350.00,0.0


In [8]:
combined = combined.drop(['no','Unnamed: 0','lat_hdb','lng_hdb','lng_mrt', 'lat_mrt', 'MRT_ID'], axis=1)

In [9]:
#combined = combined.drop(['no','Unnamed: 0','Unnamed: 0.1','lat_hdb','lng_hdb','lng_mrt', 'lat_mrt', 'MRT_ID', ], axis=1)
combined = combined.rename(columns = {'line': 'mrt_line', 'distance': 'hdb_bus_dist'})

## Adjusting for inflation 
to be done

In [10]:
#https://eservices.mas.gov.sg/statistics/calculator/GoodsAndServices.aspx
    
def adjustment(oldprice, date):
    if "2022" in date:
        return 0.96*oldprice
    elif "2021" in date:
        return 1.01*oldprice
    elif "2020" in date:
        return 1.03*oldprice
    elif "2019" in date:
        return 1.02*oldprice
    elif "2018" in date:
        return 1.01*oldprice
    elif "2017" in date:
        return oldprice
    

combined['real_price'] = combined.apply(lambda row: adjustment(row.resale_price, row.month), axis=1)
combined['real_price_persqm'] = combined.apply(lambda row: adjustment(row.sgd_persqm, row.month), axis=1)

## making remaining lease a continuous variable 


In [12]:
def conversion(remaining):
    total = float(remaining[:2])
    if len(remaining)>10:
        total += float(remaining[9:11])/12.0
    return total
        

combined['lease_remaining'] = combined.apply(lambda row: conversion(row.remaining_lease), axis=1)

In [13]:
combined.columns

Index(['month', 'town', 'flat_type', 'blk_no', 'street', 'storey_range',
       'floor_area_sqm', 'flat_model', 'lease_commence_date',
       'remaining_lease', 'resale_price', 'max_floor_lvl', 'year_completed',
       'residential', 'commercial', 'market_hawker', 'miscellaneous',
       'multistorey_carpark', 'precinct_pavilion', 'bldg_contract_town',
       'total_dwelling_units', '1room_sold', '2room_sold', '3room_sold',
       '4room_sold', '5room_sold', 'exec_sold', 'multigen_sold',
       'studio_apartment_sold', '1room_rental', '2room_rental', '3room_rental',
       'other_room_rental', 'building', 'addr', 'Postal', 'SUBZONE_NO',
       'SUBZONE_N', 'SUBZONE_C', 'PLN_AREA_N', 'PLN_AREA_C', 'REGION_N',
       'REGION_C', 'MRT_NAME', 'mrt_line', 'mahattan_distance', 'mrt_cbd_dist',
       'mrt_cbd_time', 'hdb_cbd_distance', 'hdb_cbd_time', 'hdb_to_mrtdist',
       'sgd_persqm', 'No_Bus_Stops', 'real_price', 'real_price_persqm',
       'lease_remaining'],
      dtype='object')

# What are the variables available and what do they mean?

Continuous Variables:
`floor_area_sqm`: The size of the housing in squared meters <br><br>
`resale_price`: The resale price, not adjusted to 2017 housing price. These are the nominal prices. <br><br>
`max_floor_lvl`: The highest level in that HDB <br><br>
`mahattan_distance`: A straight line distance from the nearest MRT to the CBD MRT (Raffles Place MRT) <br><br>
`mrt_cbd_dist`: The car travelling distance between the MRT and the CBD MRT (Raffles Place MRT) <br><br>
`mrt_cbd_time`: The car travlling time between the MRT and the CBD MRT (Raffles Place MRT) <br><br>
`hdb_cbd_distance`: The car travelling distance between the HDB and the CBD MRT (Raffles Place MRT) <br><br>
`hdb_cbd_time`: The car travelling time between the HDB and the CBD MRT (Raffles Place MRT) <br><br>
`hdb_to_mrtdist`: The car travelling distance between the HDB and the MRT (Raffles Place MRT) <br><br>
`sgd_persqm`: The nominal resale price per squared meter of the HDB <br><br>
`real_price`: The resale price, adjusted to 2017 housing price, using MAS adjustment calculator <br><br>
`real_price_persqm`: The resale price per squared meter, adjusted to 2017 housing price, using MAS adjustment calculator <br>
`lease_remaining`: The number of years in decimal of lease remaining <br><br>
`No_Bus_Stops`: Number of bus stops in a 200m radius <br><br>

<br>

`'total_dwelling_units', '1room_sold', '2room_sold', '3room_sold','4room_sold', '5room_sold', 'exec_sold', 'multigen_sold','studio_apartment_sold', '1room_rental', '2room_rental', '3room_rental','other_room_rental',` <br>
the above variables were provided in the github and state how many dwellings were in the HDB, how many types of each variables were sold in the HDB. While these variables are left in the dataframe for exploratory analysis, it may not significantly affect the HDB prices. 
<br><br><br>

Categorical variables:
`month`: The month in which the HDB house was resold <br><br>
`town`: The town where the HDB is <br><br>
`flat_type`: The number of rooms in the house <br><br>
`blk_no`: The HDB Block number <br><br>
`street`: The street in which the HDB is located <br><br>
`storey_range`: The range of storeys sold. <br><br>
`flat_model`: Model A vs Improved <br><br>
`lease_commence_date`: The official start of lease for the HDB <br><br>
`remaining_lease`: The remaining lease for the HDB as a 'x years and y months' format <br><br>
`year_completed`: Year of build <br><br>
`residential`: Whether it is residential <br><br>
`commercial`: whether there are any shops in the HDB <br><br>
`market_hawker`: Whether there is a market or hawker in the HDB itself <br><br>
`miscellaneous`: Any misc facilities <br><br>
`multistorey_carpark`: Whether there is an in built carpark <br><br>
`precinct_pavilion`: Whether there is a pavilion <br><br>
`bldg_contract_town`: The tender (unsure variable to be checked) <br><br>
`building`: Whether it belongs to a center company <br><br>
`SUBZONE_NO`: The ID of the sub zone (dropped) <br><br>
`mrt_line`: the Line in which the nearest MRT belongs too
`SUBZONE_N`: The subzone <br><br>
`SUBZONE_C`: The subzone code (dropped) <br><br>
`PLN_AREA_N`: The planning area <br><br>
`PLN_AREA_C`: The planning area code (dropped) <br><br>
`REGION_N`: The region name <br><br>
`REGION_C`: The region code (dropped) <br><br>
`MRT_NAME`: Name of the nearest MRT <br><br>

In [14]:
combined = combined.drop(['SUBZONE_C', 'PLN_AREA_C', 'REGION_C'], axis = 1)

## Comparison between types of HDBs
We need to recognise that some HDBs have multistory carparks in the lower levels while others do not. 
Some HDBs have hawkers in built, others have commercial shops like value dollar shops. 

So what are the HDB facilities? 
1. Commercial!
2. Multistory carpark!
4. Precint_pavillion!

what are the combination available in the given dataset?

1. Pure residential 
2. Residential and Commercial Only
3. Residential, Commercial with carpark (6 rows only)
4. Residential and pavilion

In [15]:
residential = combined.loc[(combined['residential']=="Y") & (combined['commercial']=="N") & (combined['multistorey_carpark']=="N") & (combined['precinct_pavilion']=="N") & (combined['miscellaneous']=="N")]
res_and_comm = combined.loc[(combined['residential']=="Y") & (combined['commercial']=="Y") & (combined['multistorey_carpark']=="N") & (combined['precinct_pavilion']=="N")]
res_comm_car = combined.loc[(combined['residential']=="Y") & (combined['commercial']=="Y") & (combined['multistorey_carpark']=="Y") & (combined['precinct_pavilion']=="N")]
res_pav = combined.loc[(combined['residential']=="Y") & (combined['commercial']=="N") & (combined['multistorey_carpark']=="N") & (combined['precinct_pavilion']=="Y")]
residential.columns

Index(['month', 'town', 'flat_type', 'blk_no', 'street', 'storey_range',
       'floor_area_sqm', 'flat_model', 'lease_commence_date',
       'remaining_lease', 'resale_price', 'max_floor_lvl', 'year_completed',
       'residential', 'commercial', 'market_hawker', 'miscellaneous',
       'multistorey_carpark', 'precinct_pavilion', 'bldg_contract_town',
       'total_dwelling_units', '1room_sold', '2room_sold', '3room_sold',
       '4room_sold', '5room_sold', 'exec_sold', 'multigen_sold',
       'studio_apartment_sold', '1room_rental', '2room_rental', '3room_rental',
       'other_room_rental', 'building', 'addr', 'Postal', 'SUBZONE_NO',
       'SUBZONE_N', 'PLN_AREA_N', 'REGION_N', 'MRT_NAME', 'mrt_line',
       'mahattan_distance', 'mrt_cbd_dist', 'mrt_cbd_time', 'hdb_cbd_distance',
       'hdb_cbd_time', 'hdb_to_mrtdist', 'sgd_persqm', 'No_Bus_Stops',
       'real_price', 'real_price_persqm', 'lease_remaining'],
      dtype='object')

In [19]:
def change_flooring(range):
    val = int(range[0:3]) + 1
    
    return val
        

residential['Floor_Height'] = residential.apply(lambda row: change_flooring(row.storey_range), axis=1)
residential

,month,town,flat_type,blk_no,street,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,...,mrt_cbd_time,hdb_cbd_distance,hdb_cbd_time,hdb_to_mrtdist,sgd_persqm,No_Bus_Stops,real_price,real_price_persqm,lease_remaining,Floor_Height
0,2017-03,BUKIT BATOK,4 ROOM,383,BT BATOK WEST AVE 5,07 TO 09,92.0,Model A,2005,86 years 11 months,...,23.801667,19015.7,22.508333,2885.65,4836.96,0.0,445000.0,4836.9600,86.916667,8
1,2017-04,BUKIT BATOK,4 ROOM,383,BT BATOK WEST AVE 5,19 TO 21,90.0,Model A,2005,86 years 11 months,...,23.801667,19015.7,22.508333,2885.65,5143.20,0.0,462888.0,5143.2000,86.916667,20
2,2017-05,BUKIT BATOK,4 ROOM,383,BT BATOK WEST AVE 5,01 TO 03,90.0,Model A,2005,86 years 10 months,...,23.801667,19015.7,22.508333,2885.65,4388.89,0.0,395000.0,4388.8900,86.833333,2
3,2017-09,BUKIT BATOK,5 ROOM,383,BT BATOK WEST AVE 5,10 TO 12,115.0,Improved,2005,86 years 05 months,...,23.801667,19015.7,22.508333,2885.65,4782.61,0.0,550000.0,4782.6100,86.416667,11
4,2017-12,BUKIT BATOK,4 ROOM,383,BT BATOK WEST AVE 5,10 TO 12,90.0,Model A,2005,86 years 03 months,...,23.801667,19015.7,22.508333,2885.65,4588.89,0.0,413000.0,4588.8900,86.250000,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184928,2018-05,CHOA CHU KANG,4 ROOM,152,JLN TECK WHYE,07 TO 09,101.0,Model A,1998,78 years 08 months,...,22.730000,18297.9,23.383333,23.24,3287.13,0.0,335320.0,3320.0013,78.666667,8
184929,2018-08,CHOA CHU KANG,4 ROOM,152,JLN TECK WHYE,13 TO 15,101.0,Model A,1998,78 years 05 months,...,22.730000,18297.9,23.383333,23.24,3712.87,0.0,378750.0,3749.9987,78.416667,14
184930,2018-11,CHOA CHU KANG,4 ROOM,152,JLN TECK WHYE,01 TO 03,100.0,Model A,1998,78 years 02 months,...,22.730000,18297.9,23.383333,23.24,3250.00,0.0,328250.0,3282.5000,78.166667,2
184931,2020-03,CHOA CHU KANG,4 ROOM,152,JLN TECK WHYE,04 TO 06,100.0,Model A,1998,76 years 10 months,...,22.730000,18297.9,23.383333,23.24,3350.00,0.0,345050.0,3450.5000,76.833333,5


In [ ]:
sb.lmplot(x='hdb_cbd_distance', y='real_price', hue='flat_type', data=residential, scatter_kws={'s':50})

# Add labels and title
plt.xlabel('Total Bill')
plt.ylabel('Tip')
plt.title('Tips by Total Bill, Colored by Sex')

# Show plot
plt.show()

In [ ]:
def printgraph(x:str,y:str):
    X=residential[[x]]
    Y=residential[[y]]
    
    titlestr = x + ' against ' + y + ' for purely residential HDBs'

    regressor = LinearRegression()
    regressor.fit(X,Y)
    y_pred = regressor.predict(X)
    #plt.scatter(X, Y, color = 'blue',s=2)
    sb.scatterplot(x='hdb_cbd_distance', y='floor_area_sqm', hue='flat_type', data=residential,s=10)
    plt.plot(X, regressor.predict(X), color = 'red', linewidth=5)
    plt.title(titlestr)
    plt.xlabel(x)
    plt.ylabel(y)
    plt.show()
    
    print("\n\n\n")
    X=res_and_comm[[x]]
    Y=res_and_comm[[y]]
    
    titlestr = x + '  against ' + y + ' for residential, commercial HDBs'

    regressor = LinearRegression()
    regressor.fit(X,Y)
    y_pred = regressor.predict(X)
    plt.scatter(X, Y, color = 'red', )
    plt.plot(X, regressor.predict(X), color = 'blue')
    plt.title(titlestr)
    plt.xlabel(x)
    plt.ylabel(y)
    plt.show()
    
    print("\n\n\n")
    X=res_comm_car[[x]]
    Y=res_comm_car[[y]]
    
    titlestr = x + '  against ' + y + ' for residential, commercial, carpark HDBs'

    regressor = LinearRegression()
    regressor.fit(X,Y)
    y_pred = regressor.predict(X)
    plt.scatter(X, Y, color = 'pink', )
    plt.plot(X, regressor.predict(X), color = 'blue')
    plt.title(titlestr)
    plt.xlabel(x)
    plt.ylabel(y)
    plt.show()
    
    print("\n\n\n")
    X=res_pav[[x]]
    Y=res_pav[[y]]
    
    titlestr = x + '  against ' + y + ' for residential, pavilion HDBs'

    regressor = LinearRegression()
    regressor.fit(X,Y)
    y_pred = regressor.predict(X)
    plt.scatter(X, Y, color = 'red', )
    plt.plot(X, regressor.predict(X), color = 'blue')
    plt.title(titlestr)
    plt.xlabel(x)
    plt.ylabel(y)
    plt.show()
    
def printmenu():
    llist = ['year_completed','floor_area_sqm', 'resale_price', 'max_floor_lvl','total_dwelling_units', '1room_sold', '2room_sold', '3room_sold',
       '4room_sold', '5room_sold', 'exec_sold', 'multigen_sold',
       'studio_apartment_sold', '1room_rental', '2room_rental', '3room_rental',
       'other_room_rental', 'mahattan_distance', 'mrt_cbd_dist',
       'mrt_cbd_time', 'hdb_cbd_distance', 'hdb_cbd_time', 'hdb_to_mrtdist',
       'sgd_persqm', 'real_price', 'real_price_persqm', 'lease_remaining','No_Bus_Stops']
    
    print("Below are the variables: ")
    count = 0
    for item in llist:
        print("item ", count, ": ", item)
        count +=1
        
    x_var = int(input("Choose x variable (type in the item number only!): \n"))
    y_var = int(input("Choose y variable (type in the item number only!): \n"))
    
    
    return llist[x_var], llist[y_var]

# Continuous Variables 

## User guide to using the below cell:

1. press shift + enter to run the cell. 
2. Enter the item number you wish to compare. Only numbers allowed


In [ ]:
x_var, y_var = printmenu()
printgraph(str(x_var), str(y_var))

# Categorical Variables

The below will only allow y variable to be sgd_persqm or resale price!

## User guide to using the below cell:

1. press shift + enter to run the cell. 
2. Enter the item number you wish to compare. Only numbers allowed


In [ ]:
def printgraph2(X:str,Y:str):
    titlestr = X + '  against ' + Y + ' for residential only HDBs'
    f = plt.figure(figsize=(10,10))
    ax = sb.boxplot(x=X, y=Y, data = residential).set(title = titlestr)
    plt.xticks(rotation=70)
    print("\n\n")
    
    titlestr = X + '  against ' + Y + ' for residential and commercial HDBs'
    f = plt.figure(figsize=(8,8))
    sb.boxplot(x=X, y=Y, data = res_and_comm).set(title = titlestr)
    plt.xticks(rotation=70)
    print("\n\n")
    
    titlestr = X + '  against ' + Y + ' for residential, commercial, carpark HDBs'
    f = plt.figure(figsize=(8,8))
    sb.boxplot(x=X, y=Y, data = res_comm_car).set(title = titlestr)
    plt.xticks(rotation=70)
    print("\n\n")
    
    
    titlestr = X + '  against ' + Y + ' for residential and pavilion HDBs'
    f = plt.figure(figsize=(8,8))
    sb.boxplot(x=X, y=Y, data = res_pav).set(title = titlestr)
    plt.xticks(rotation=70)
    
    
    
def printmenu2():
    llist = ['Floor_Height','month', 'town', 'flat_type', 'blk_no', 'street', 'storey_range',
       'flat_model', 'lease_commence_date',
       'remaining_lease', 'year_completed','bldg_contract_town',
       'building', 'SUBZONE_NO', 'mrt_line',
       'SUBZONE_N', 'SUBZONE_C', 'PLN_AREA_N', 'PLN_AREA_C', 'REGION_N',
       'REGION_C', 'MRT_NAME']
    
    print("Below are the variables: ")
    count = 0
    for item in llist:
        print("item ", count, ": ", item)
        count +=1
        
    x_var = int(input("Choose x variable (type in the item number only!): \n"))
    
    yvar = ['resale_price','sgd_persqm','real_price', 'real_price_persqm', 'floor_area_sqm']
    
    count = 0
    for item in yvar:
        print("item ", count, ": ", item)
        count +=1
    
    y_var = int(input("Choose y variable (type in the item number only!): \n"))
    
    
    return llist[x_var], yvar[y_var]

In [ ]:
x_var, y_var = printmenu2()

printgraph2(x_var,y_var)

In [17]:
residential.to_csv('residential.csv',index_label = True)